<a href="https://colab.research.google.com/github/paulodowd/EMATM0054_53_23-24/blob/main/Labsheets/Core/L4_LineFollowing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Labsheet 4: Line Following

In previous labsheets we have:

- Familiarised with the Arduino IDE, Sketch files, uploading to the 3Pi+, and utilising the Serial interface
- Implemented basic operation of the 3Pi+ motors, and encapsulated this within safe and confident function(s), and utilised a `class`.
- Implemented basic read functionality of the 3 central IR Line sensors facing the work surface, and encapsulated this within function(s).
- Encapsulated the code for the line sensors within a `class`, refactoring the code so we can be confident there are no errors and bugs.
- Explored the use of `millis()` to approximate task-scheduling on the 3Pi+.

In this labsheet we will:
- Utilise prior work with the line sensors and motors to implement line following behaviour on the 3Pi+.
- Develop a **bang-bang** controller, using `logic` to control the robot.
- Implement a calibration routine for the ground sensor.  
- Develop a **weighted-measurement** of the line position beneath the robot, and provide this as input to a `proportional controller` to control the robot motors.


If you have prior experience with programming, you may wish to skip the section for the bang-bang controller.  The bang-bang controller is generally inefficient, but provides a good introduction to the concepts of providing feedback control to the 3Pi+.  It is still recommended to read through the bang-bang controller section regardless.


<hr><br><br><br><br>

# Line Following, in Overview

<p align="center">
<br>
<img src="https://github.com/paulodowd/EMATM0054_53_23-24/blob/main/Images/3PI_BottomAnnotated.png?raw=true">
<br>
</p>

In this labsheet, we will continue to work with just the 3 central ground sensors, labelled `DN2`, `DN3` and `DN4` on the 3PI+ and in the documentation.  The black vinyl tape provided with your 3Pi+ fits between the sensors `DN2` and `DN4`.  


<p align="center">
<br>
<img width="50%"  src="https://github.com/paulodowd/EMATM0054_53_23-24/blob/main/Images/3Pi_SensorsTape.jpeg?raw=true">
<br>
</p>



We can quickly sketch out a plan to produce line following behaviour from the knowledge we have so far.  In the illustration below, there are 3 examples of the robot in different scenarios with respect to the line:

<p align="center">
<br>
<img src="https://github.com/paulodowd/EMATM0054_53_23-24/blob/main/Images/Webots_LineFeedback.png?raw=true">
<br>
</p>

We can quickly draft the following logic:

- In **`scenario (A)`**, the central sensor is activated by the black line, and so the robot can continue with it's normal direction of travel.  This is the ideal scenario for line following.  Here, the ideal is to keep the black line between the inactivated outside sensors.  Note that, whilst we (humans) can observe the robot _will_ leave the line, presently the robot does not have this information.

- In **`scenario (B)`**, the robot has veered off the line to the **`left`**, causing the right-most sensor to become activated.  To bring the line back under the centre sensor, the appropriate feedback is to **`turn right`**.

- In **`scenario (C)`**, the robot has veered off the line to the **`right`**, causing the left-most sensor to become activated.  To bring the line back under the centre sensor, the appropriate feedback is to **`turn left`**.

In essence, we can write a controller to keep the black line between the left and right sensors.  Note, there are other ways to utilise this sensor to follow the line.  

**Note:** It is possible to produce quite convincing line following that is actually operating in an error condition.  Make sure you take your time to **validate** how your code is producing the observable behavior of the robot.  Remember that you have tools at your disposal to help you when debugging:
- Audible Beeping
- Serial Monitor
- 3 LEDs on the 3Pi+ circuit board.
  - **Note:** Remember that the LEDs labelled `Tx` and `Rx` are `ACTIVE LOW` - this means setting them `LOW` is equivalent to `ON`, and setting them `HIGH` is equivalent to `OFF`.  
- Peers to talk to engaged in the same tasks (discussion helps a lot!)



## Exercise 1: Line Sensors

Feel free to discuss these questions with your peers.  It is a valuable skill to be able to construct **hypotheses** prior to working with the robot:

1. **Validate:** Find the line sensors on the robot.  
  - Check the printed circuit board (PCB) for the white text labelling the sensors.
  - Identify `DN1`,`DN2`,`DN3`,`DN4`,`DN5`.  When you put the robot correctly on the surface, identify where each of these are located.  We want to ensure we understand which is on the left and which is on the right.  

1. **Hypothesise**: Given your prior experience of the line-sensors, what do we expect the time-values of the 3 ground sensors to be on white and black surfaces?
  - in the above example, we have considered the central sensor to be "active" on the line, but we could also define this in inverted terms.  
  
2. **Hypothesise**: What motion would you expect to observe in the robot motion if the feedback-signal was inverted?
  - is there a case where line following can be achieved with an inverted feedback signal?

3. **Hypothesise**: An exceptional case not caught with the above logic would be if all three ground-sensors were detecting black simultaneously.  This can happen even though the black line is narrower than the gap between the two outer-most sensors.  Under what condition might this occur?

4. **Hypothesise**: What would be appropriate feedback responses for other exceptional (non-defined) cases of the sensor activation?

5. **Hypothesise**: If your robot was to calculate a performance score within `loop()` for line-following (a **metric**) as it operated:
  - what **`proprioceptive`** information could be used?
  - what **`exteroceptive`** information could be used?

<hr><br><br><br><br>

# Bang-Bang Controller

<p align="center">
<img src="https://github.com/paulodowd/EMATM0054_53_23-24/blob/main/Images/3PI_PhotoShortLine.jpg?raw=true">
</p>

The most intuitive form of controller we can write for line following is a **`bang-bang`** controller.  A bang-bang controller has this name because the control decisions tend to be sudden and dramatic (bang!), moving between discrete states (bang<->bang).  

A consequence of this can be that there is no gradation in behaviour, or the behaviour looks jerky.  Jerky motion can be a problem, because it can cause the wheels to slip on the surface.  We'll discuss this again in a later labsheet on Odometry.  

Usually, a bang-bang controller is not a good controller to use, but it is a good place to start if you have not programmed before.  It is possible to complete the coursework for Assessment 1 only with a Bang-Bang controller - and it is acceptable to do this.  Later, you will investigate using PID to control the motors, and you can achieve much higher performance this way.

If you have not done much programming before, a Bang-Bang controller will help to develop a sense of how `conditional statements` can be used to branch code, to change the flow of your program, and so change the observable robot behaviour.  

A bang-bang controller is built using a series of **`if()`** statements.  We will use the `if()` statement to check if a condition is true, and if so, effect a specific behaviour.  

We can interpret the above section and write some **`pseudo-code`**:

```c

void loop() {
  
  // Greater than or equal to
  if( left_sensor >= active_threshold ) {
      // Turn right.

  // Greater than or equal to
  } else if( right_sensor >= active_threshold ) {
      // Turn left

  } else {
      // Move forwards.
  }


}
```

The above example would need significant changes to work with your code.  From Labsheet 3, you will have implemented a sensor reading function.  From Labsheet 2, you will have implemented some motor command functions.  



## Exercise 2: Implement a Bang-Bang Controller

1. Is a bang-bang controller **`open-loop`** or **`closed-loop`** control?  What is the difference?

2. Using your prior experience taking readings from the ground sensors, decide appropriate thresholds for when each sensor can be considered `active` or `inactive`:
  - decide whether large time measurements indicate the presence of a line or not.
  - decide on some time measurement values to use as a threshold. For example, at what value would we say the line was present?
  - which sensor states will be used to build the bang-bang control logic discussed above?  You could use individual sensor states, or a combination of them.  
  - are these threshold values the same for `DN2`, `DN3` and `DN4`?  Remember that you can inspect the values of your sensors using `Serial.print()` and `Serial.println()`.
  - remember that you can use the `Serial Plotter` built into the Arduino IDE to look at multiple sensor values simultaneously.
  - it is recommended to use `#define` statements to set these values at the top of your program.  This way, they will be easy to find and change.
  

3. Implement the discussed bang-bang controller logic within your `loop()`:
  - to begin with, place your robot on the line.  Do not expect your robot to be able to find the line if it is not on the line to start with.
  - use the black vinyl tape to create a short black line on your work surface.
  - it is recommended you start with `slow` or `low` motor velocities.
  - to begin with, do not implement forward motion.  Instead, work only with turning on the spot (rotation).  Does your robot turn the correct way to get back to central on the line?
  - check that your `feedback signal` turns your robot in the appropriate directions with respect to sensor activation.
  - investigate what happens when your line is placed into difficult places with respect to the line - are any of these a problem you need to address?
  - once you are confident rotation is working properly, implement forward motion.
  - **help**: remember your conditional statement can use:
    - `<`  less than
    - `<=` less than or equal to
    - `==` equal to
    - `>=` greater than or equal to
    - `>`  greater than
    - `!=` not equal to
  - **help**: what is the functional difference between the two code examples immediately below?
  
```c
  // Example 1
  if( ) {

  }
  if( ) {

  }

  // Example 2
  if( ) {

  } else if( ) {

  }
```

4. Does your robot conduct `turn` and `move fowards` operations seperately?  
  - Can these be integrated so that the robot does not stop moving forwards?
    - it may be useful to use your bang-bang logic to set the value of left and right `pwm` or `power` variables, which are then used to command the motors once after the logic.
  - How is performance effected with turning and moving forwards combined?
  - Moving slowly might increase the general reliability of the line following behaviour.  As a thought experiment, what other hypothetical **`task requirements`** would make fast forward speed desirable for the robotic system?
  - What is the quickest forward speed you can utilise and still achieve reliable line-following?
  - If you have not done so already, experiment with more challenging line shapes, such as corners and curves.  

5. What information about the line does the robot have when no sensors detect the black surface?
  - When might this circumstance occur?  Consider the line map provided for the line following challenge.
  - What would be an appropriate response in this condition?
  - What other information is available to the robot that might be useful?

6. Write a function to simply confirm if the robot is on a black line.  The function should report a `true` or `false` value when called.  For example, a function named `lineDetected()` (returning true or false).
  - is there a reason to discriminate between which of the sensors is on a black surface, or can it be any of the 3?  Explain your reasoning, adjust the function if necessary.

7. Use the above function to allow your robot to start off the line and drive forwards until it meets the line.  Once it is on the line, it should activate (begin calling) the bang-bang controller.
  - Consider using a `global` variable and an `if()` statement to switch the robot between these two behaviours, or `states` of operation.

8. Start your robot off the line, and allow it to travel forward to join and follow the line.  Currently, what is the most extreme <a href="https://en.wikipedia.org/wiki/Angle_of_incidence_(optics)">angle of incidence</a> where your controller can still successfully begin line following?
  - if you were to create a results table of different angles when joining the line, how could you quantify the reliability of the controller?


As you work further on your Bang-Bang controller, it may be worth reviewing the idea of a system block diagram.  The below is a minimum viable product (the fewest parts we need for a working solution):

<p align="center">
<img src="https://github.com/paulodowd/EMATM0054_53_23-24/blob/main/Images/MVP.png?raw=true">
</p>

And we can imagine a more complete block diagram featuring a bang-bang controller:

<p align="center">
<img src="https://github.com/paulodowd/EMATM0054_53_23-24/blob/main/Images/BangBang.png?raw=true">
</p>

In the above, we have included Kinematics to estimate location, which the 3Pi+ robot would need to complete Component 6 of Assessment 1.

<hr><br><br><br><br>

# Weighted-Measurement

The bang-bang controller has the disadvantage that it uses `hard-coded` (fixed) parameter values to determine behaviour. This causes the controller to switch between discrete states, which means it is not very good at responding to continuous change.  For example, we might observe this as jerking motion when the robot follows a curved line.  The curve represents a continuous change, but the robot can only turn in increments of a fixed amount.  


In this section we will investigate a `weighted-measurement`.  In summary, we will look at the level of activation of the left and right sensors of the middle triplet, `DN2` and `DN4`, to attempt to estimate the location of the line beneath the robot.  We can think of this as "weighing one sensor activation against another" to approximate the position of the line.

<p align="center">
<img width="75%" src="https://raw.githubusercontent.com/paulodowd/EMATM0054_53_23-24/main/Images/3Pi_weighted_measurement.png?raw=true">
</p>

The following pseudocode describes how we can construct this measurement:

<p align="center">
<img src="https://github.com/paulodowd/EMATM0054_53_23-24/blob/main/Images/Algo5_WeightedMeasument1.png?raw=true">
</p>

In this section, we assume that to "be on the line" would result in a signal with value `0.0`, whilst to be to the left or right of the line would create values towards the extremes of either `-1.0` or `+1.0`.  This is a useful way to condition or process the signal because:
- a measurement of 0.0 represents no turning required (no feedback)
- a measurement less than 0 (negative) could mean turn left
- a measurement more than 0 (positive) could mean turn right

Note that, when we use "left" or "right", this depends on how you view the operation of your robot, and how you perform the calculation.  

Processing the sensors to provide a signal in the range of `[-1.0 : +1.0]` has the benefit of supporting a numerically continuous approach to the robot control.  Remember that:
- any value multiplied by 0 equals 0.
- any value multiplied by 1.0 remains the same value.
- any value multiplied by a value in the intermediate range [0.0:1.0] is scaled.
  - For example, `10.0 * 0.8 = 8.0`
  - For example, `20.0 * -0.2 = -4.0`.

Therefore, we could decide the maximum power or rate that the robot could turn (e.g., a PWM value of 100), and then allow a the weighted-measurement of the line $W$ to scale this.  Therefore, the more the robot was off the line, the more it would turn.  Which way to turn, direction, would be implicitly captured by the sign of the value $W$.  

For example, if the weighted measument value was `0.3`, a maximum PWM of 100 to turn would be scaled to `33`.  Or, if the weighted measument value was `-0.15`, a maximum PWM to turn of 100 would be scaled to `-15`.

To produce an output signal in the range `[-1.0:+1.0]` with have to first **normalise the sensor readings**.  In the above pseudocode (algorithm 5, line 2), we construct a normalised value by dividing each sensor reading $S_{n}$ by the sum total of both sensor readings.  We can think of this as working out what proportion of the total activation each sensor represents.  This is also similar to working out what _percentage_ of the sum total each sensor represents.  However, we do not multiple by 100 to create a standard percentage (i.e., per-centage, "per 100") - so this isn't a percentage.  Therefore, each normalised value will be in the range `[0.0:1.0]`.

Because we want an output with a value in range `[-1.0 : +1.0]`, this means the total range of the output signal $W$ has an absolute value of `2.0`.  So far, our normalisation means our normalised values will always add back up to a value of `1.0` - remember, we have worked them out as a propotion of a total, where the total will be 1.0.  Therefore, Algorithm 5, Line 5 multiplies by a weighting of `2.0` to each normalised value $N_{n}$, which will allow the output to swing both positive and negative when they are subtracted from each other on line 7.  






## Exercise 3: Weighted-Line Measurement

1. In Algorithm 5 above, line 2 has $Sum ← R_{1} + R_{3}$, but the comment refers to `DN2` and `DN4`.  Why might the enumeration be different between $R$(readings) and `DN` (sensor number)?

2. Implement Algorithm 5 onto your 3Pi+ robot.  
  - Remember to keep things as simple as possible to begin with.  At this stage, don't activate the motors for your robot.  Instead, your goal should be to use the Serial Plotter to see how your value of $W$ is changing with respect to the robot position on the line.

  - **Validate:** Use `Serial.print()` to output the values of your line sensors.
  - **Validate:** Work methodically, check that you calculate the sum correctly,and that your resultant normalised values are sensible.
  - If you have some doubts, you can augment `Serial.print()` to show more decimal places by writing something like `Serial.println( variable, 4)` - this would produce 4 decimal places.  
  - **Validate:** Finally, use the Serial Plotter to inspect the value of the Weighted-Mesaurement $W$.  
  

# Exercise 4: Line Following, Weighted-Line Measurement

<p align="center">
<img src="https://github.com/paulodowd/EMATM0054_53_23-24/blob/main/Images/Algo6_LineFollowingBehaviour.png?raw=true">
</p>

The above pseudocode is written as if it were a function that is called iteratively.  This means it does not have anything like `while( OnLine == true)`.  Therefore, the presented pseudocode is **non-blocking**.

We would expect the above pseudocode function to be called, a decision made, the function `return` issued, and then the main Arduino `loop()` function to run again.   **It is strongly recommended you attempt follow the same approach.**  

As we add more functionality to your robot, using `while()` or `for()` loops to control the flow of your code (**blocking code**) is likely to become a problem for you.

1. Implement Algorithm 6 above.
  - **Hypothesise:** To start, set `BiasPWM` to `0.0`.  What would be the effect of this?
  - **Hypothesise:** Lines 6&7 of Algorithm 6 have a `-` and `+` symbol, but are otherwise the same. What effect does this have, why it is necessary?
  - **Validate:** Check that your robot is centering itself on the line.
    - If your robot is turning the wrong way, you can either alter line 7 of Algorithm 5, or lines 7&8 of Algorithm 6.  
  - **Validate:** Line 8 of Algorithm 6 assumes a function that can receive a negative PWM value and set the correct direction pins on your 3Pi+ robot. Have you implemented this?
  - Line 4 tests to see if the robot is currently on the line.  If you have not implemented this functionality, it will be helpful to do so.

2. Set `BiasPWM` to a positive value, such as 30 in the pseudocode.  
  - **Research:** Experiment with different values of `BiasPWM`.  What is the maximum speed at which your robot can still follow a line?
    - Do particular line shapes present more difficulty?
  - **Research:** Co-vary `MaxTurnPWM` against `BiasPWM`.  It will help to try and be systematic.  For example:
    - Does a low `BiasPWM` and high `MaxTurnPWM` offer a benefit?
    - Does a high `BiasPWM` and low `MaxTurnPWM` offer a benefit?

3. Lines 9,10,11 represent what the robot should do if it comes off the line.  Currently, this may be very abrupt - your robot might stop suddenly when it has only come off the line by a tiny amount.
  - **Hypothesise:** how can this section of the code be improved so that the robot is given an opportunity to rediscover or turn to the line?
    - **Note:** here, either elapsed time (see labsheet 3) or a variable to hold an incrementing count might be useful.


As you continue to work on your Weighted-Measurement Line following, it is worthwhile to recall the system block diagram.  The numerical approach of the Weighted-Measurement is particularly suitable for PID control of the motors:

<p align="center">
<img src="https://github.com/paulodowd/EMATM0054_53_23-24/blob/main/Images/SystemBlockDiagram.png?raw=true">
</p>